# Final Real Estate AI Pipeline

Optimized for Business Value & High Accuracy.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, classification_report
import xgboost as xgb
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

## 1. Data Cleaning & Understanding

In [ ]:
# Load Data
import pandas as pd

df = pd.read_csv('/kaggle/input/housedata/data.csv')

# Convert date
df['date'] = pd.to_datetime(df['date'])

# Keep only allowed columns and drop missing values
allowed_cols = [
    'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
    'floors', 'waterfront', 'view', 'condition', 'sqft_above', 'sqft_basement', 
    'yr_built', 'yr_renovated', 'street', 'city', 'statezip', 'country'
]
df = df[allowed_cols].dropna()

# Remove outliers using IQR method
q_low = df["price"].quantile(0.01)
q_hi  = df["price"].quantile(0.99)
df_clean = df[(df["price"] > q_low) & (df["price"] < q_hi)]

print(f"Original: {df.shape}, Cleaned: {df_clean.shape}")

# Replace df with cleaned version
df = df_clean.copy()


## 2. Advanced Feature Engineering

In [ ]:
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler


# 1. City Rank
city_price_map = df.groupby('city')['price'].median().sort_values().to_dict()
df['city_enc'] = df['city'].map(city_price_map)

# 2. Derived Time Features
df['house_age'] = 2026 - df['yr_built']
df['was_renovated'] = df['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)


# 3. Log Target
y = np.log1p(df['price'])

# Save Encoders / Stats for App
stats = {
    'median_price': df['price'].median(),
    'avg_price_sqft': (df['price'] / df['sqft_living']).mean(),
    'zip_map': zip_price_map,
    'city_map': city_price_map
}
joblib.dump(stats, 'market_stats.pkl')

# 4. Feature Selection
drop_cols = ['date', 'street', 'city', 'statezip', 'country']
X = df.drop(drop_cols, axis=1, errors='ignore')

# 5. Scaling
cols = X.columns
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=cols)

print("Engineering Complete. Features:")
print(X_scaled.head())


## 3. Exploratory Analysis & Business Insights

Based on dataset analysis, here are the key drivers and recommendations:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Price Drivers
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()['price'].drop('price').sort_values(ascending=False)
print("Top Drivers:\n", corr_matrix.head(5))

# Visuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.scatterplot(data=df, x='sqft_living', y='price', alpha=0.3, ax=axes[0]).set_title('Price vs SqFt')
sns.boxplot(data=df, x='view', y='price', ax=axes[1]).set_title('Price vs View')
plt.tight_layout()
plt.show()


## 4. Clustering (Unsupervised)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# Cluster using Price + Sqt 
clust_feats = df[['price', 'sqft_living']]
X_c = StandardScaler().fit_transform(clust_feats)

km = KMeans(n_clusters=3, random_state=42)
df['cluster'] = km.fit_predict(X_c)

# Map clusters to meaningful categories
means = df.groupby('cluster')['price'].mean().sort_values()
mapping = {
    means.index[0]: 'Affordable Home',
    means.index[1]: 'Family Home',
    means.index[2]: 'Luxury Villa'
}
df['category'] = df['cluster'].map(mapping)

# Plot clusters
sns.scatterplot(data=df, x='sqft_living', y='price', hue='category', palette='coolwarm')
plt.title('Market Segments')
plt.show()


## 5. Classification (Supervised)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Prepare data
X_final = X_scaled.values
y_cls = df['category'].values

# Split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_final, y_cls, test_size=0.2, random_state=42
)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_c, y_train_c)

# Save model and feature names
joblib.dump(clf, 'category_classifier.pkl')
joblib.dump(X_scaled.columns.tolist(), 'cls_features.pkl')

# Evaluation
print("Classification Report:")
print(classification_report(y_test_c, clf.predict(X_test_c)))


## 6. Price Regression (Supervised)

In [ ]:
import xgboost as xgb
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Prepare target
y_reg = y.values  # log-transformed price

# Train-test split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_final, y_reg, test_size=0.2, random_state=42
)

# Train XGBoost Regressor
print("Training XGBoost Regressor...")
model = xgb.XGBRegressor(
    n_estimators=2000,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.7,
    colsample_bytree=0.7,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train_r, y_train_r)

# Save model and features
joblib.dump(model, 'price_regressor.pkl')
joblib.dump(X_scaled.columns.tolist(), 'reg_features.pkl')

# Evaluation
preds_log = model.predict(X_test_r)
preds_dollar = np.expm1(preds_log)
y_true_dollar = np.expm1(y_test_r)

r2 = r2_score(y_true_dollar, preds_dollar)
mae = mean_absolute_error(y_true_dollar, preds_dollar)

print(f"Final R2 Score: {r2:.4f}")
print(f"Final MAE: ${mae:,.0f}")


## 7. App Generation

In [ ]:
# Code for Streamlit App 
app_source = 'import streamlit as st\nimport pandas as pd\nimport joblib\nimport numpy as np\nimport xgboost as xgb\nimport os\nfrom groq import Groq\n\n# Load assets\n@st.cache_resource\ndef load_models():\n    clf = joblib.load(\'category_classifier.pkl\')\n    reg = joblib.load(\'price_regressor.pkl\')\n    cls_feats = joblib.load(\'cls_features.pkl\')\n    reg_feats = joblib.load(\'reg_features.pkl\')\n    market_stats = joblib.load(\'market_stats.pkl\')\n    return clf, reg, cls_feats, reg_feats, market_stats\n\ndef get_explanation(features, price, category, stats):\n    median = stats[\'median_price\']\n    prompt = f"""\n    Act as a real estate analyst.\n    Data:\n    - Valuation: ${price:,.0f} ({category})\n    - Specs: {features[\'sqft_living\']}sqft, {features[\'bedrooms\']}bed, Zip-Rank {features[\'zip_rank\']:.2f}\n    - Market Median: ${median:,.0f}\n    \n    Output strictly:\n    1. Reason for value (1 sentence).\n    2. Verdict (Fair/High/Low).\n    """\n    try:\n        api_key = st.secrets.get("GROQ_API_KEY") or os.getenv("GROQ_API_KEY")\n        if not api_key: return "⚠️ Groq Key missing."\n        \n        client = Groq(api_key=api_key)\n        completion = client.chat.completions.create(\n            model="llama-3.3-70b-versatile",\n            messages=[{"role": "user", "content": prompt}],\n            max_tokens=60, temperature=0.3\n        )\n        return completion.choices[0].message.content\n    except Exception as e:\n        return f"Insight unavailable."\n\n# Main App\nst.set_page_config(page_title="RE AI", layout="wide")\ntry:\n    clf, reg, cls_cols, reg_cols, stats = load_models()\n    city_map = stats.get(\'city_map\', {})\n    zip_map = stats.get(\'zip_map\', {})\nexcept:\n    st.error("Models not found. Please train models first using the notebook.")\n    st.stop()\n\nst.title("Real Estate Valuation AI 🏡")\n\n# Sidebar\nst.sidebar.header("Details")\n\n# Location Inputs (Encoded)\nc1, c2 = st.sidebar.columns(2)\n# Safety check for maps\nif not city_map: city_map = {\'Unknown\': 0}\nif not zip_map: zip_map = {0: 0}\n\ncity = c1.selectbox("City", sorted(city_map.keys()))\nzipcode = c2.selectbox("Zipcode", sorted(zip_map.keys()))\n\nsqft_living = st.sidebar.number_input("SqFt Living", 300, 10000, 2000)\nbedrooms = st.sidebar.slider("Bedrooms", 0, 10, 3) \nbathrooms = st.sidebar.slider("Bathrooms", 0.0, 8.0, 2.0, 0.5) \nyr_built = st.sidebar.number_input("Year Built", 1900, 2025, 2000)\nyr_renov = st.sidebar.number_input("Year Renovated", 0, 2025, 0)\nview = st.sidebar.slider("View (0-4)", 0, 4, 0)\ncond = st.sidebar.slider("Condition (1-5)", 1, 5, 3)\nwater = st.sidebar.selectbox("Waterfront", [0, 1])\nsqft_above = st.sidebar.number_input("SqFt Above", 0, 10000, 1500)\nsqft_bsmt = st.sidebar.number_input("SqFt Basement", 0, 5000, 0)\nsqft_lot = st.sidebar.number_input("SqFt Lot", 0, 50000, 5000)\nfloors = st.sidebar.slider("Floors", 1.0, 3.5, 1.0, 0.5)\n\nif st.sidebar.button("Valuate"):\n    # Encode Input\n    city_val = city_map.get(city, 0)\n    zip_val = zip_map.get(zipcode, 0) # Median Price of Zip\n    \n    # Derived Logic (Identical to Notebook)\n    house_age = 2014 - yr_built\n    was_renovated = 1 if yr_renov > 0 else 0\n    \n    features = {\n        \'sqft_living\': sqft_living,\n        \'bedrooms\': bedrooms,\n        \'bathrooms\': bathrooms,\n        \'floors\': floors,\n        \'waterfront\': water,\n        \'view\': view,\n        \'condition\': cond,\n        \'sqft_above\': sqft_above,\n        \'sqft_basement\': sqft_bsmt,\n        \'sqft_lot\': sqft_lot,\n        \'yr_built\': yr_built,\n        \'yr_renovated\': yr_renov,\n        # Encoded Features\n        \'city_enc\': city_val,\n        \'zip_rank\': zip_val,\n        \'house_age\': house_age,\n        \'was_renovated\': was_renovated,\n        \'sqft_zip_interaction\': sqft_living * zip_val \n    }\n    \n    # DataFrame for Model\n    input_df = pd.DataFrame([features]).reindex(columns=reg_cols, fill_value=0)\n    cls_input = pd.DataFrame([features]).reindex(columns=cls_cols, fill_value=0)\n\n    c1, c2 = st.columns(2)\n    \n    try:\n        log_price = reg.predict(input_df)[0]\n        price = np.expm1(log_price)\n        cat = clf.predict(cls_input)[0]\n        \n        c1.subheader(f"Type: {cat}")\n        c2.metric("Valuation", f"${price:,.0f}")\n        \n        st.divider()\n        with st.spinner("AI analyzing..."):\n            st.info(get_explanation(features, price, cat, stats))\n            \n    except Exception as e:\n        st.error(f"Error: {e}")\n'

with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_source)
    
with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write("streamlit\npandas\nnumpy\nscikit-learn\nxgboost\njoblib\ngroq")